In [1]:
# This section is importing all libraries & data used for the project

# importing all necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# importing the training and testing data file
train = pd.read_csv("./DataFiles/CreditCard_train.csv", header=1, index_col='ID')
test = pd.read_csv("./DataFiles/CreditCard_test.csv", header=1, index_col='ID')

# From our training data, we want to split a validation set
train, val = train_test_split(train, test_size=0.1)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

train

21600 train examples
2400 validation examples
6000 test examples


LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
ID                                                                            
11652     130000    2          1         2   30      1     -2     -2     -2   
12289      30000    1          2         2   35      0      0      2      2   
9793      150000    2          2         1   28      0      0      0      0   
8526      150000    2          2         1   36      1      2      2      4   
4644      500000    2          1         1   37      0      0      0      0   
...          ...  ...        ...       ...  ...    ...    ...    ...    ...   
13037     230000    2          1         2   26     -1      0     -1     -1   
2478      160000    1          2         2   37     -1     -1      0      0   
4243       20000    1          2         1   30     -1     -1     -1     -1   
23164     170000    2          2         1   45     -1     -1      0      0   
17260     280000    2          2         1   37      0      0      0     -2   

       PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  \
ID            ...                                                        
11652     -2  ...       -915       -915      -1070         0         0   
12289     -2  ...          0          0          0      3026         0   
9793       0  ...      70371      72257      72536      5008      5000   
8526       3  ...     139097     138592     141447      9734     14500   
4644       0  ...     287998     298343     308378     10000     15000   
...      ...  ...        ...        ...        ...       ...       ...   
13037     -1  ...       2688       2688       6393     10000      2688   
2478       0  ...      29638      35474      38190     52165      5000   
4243      -1  ...        390        390          0      1473       390   
23164      0  ...       6208      10460      11143      9807      5000   
17260     -2  ...          0          0          0     10000         0   

       PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  default payment next month  
ID                                                                         
11652         0         0         0         0                           0  
12289         0         0         0         0                           0  
9793       2499      3000      2789      2439                           0  
8526          0      2000      5300      5000                           0  
4644      15000     15000     15000     15200                           0  
...         ...       ...       ...       ...                         ...  
13037      2688      2688      6393      6504                           0  
2478       7000      8000      5000      1200                           0  
4243        390       390         0       780                           0  
23164      3749      6711      4432         0                           0  
17260         0         0         0         0                           0  

[21600 rows x 24 columns]

In [3]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('default payment next month') #Our target
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe),labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [4]:
batch_size = 5 
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

### Let's illustrate the data we feed in through tensorflow dataset

In [5]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of ages:', feature_batch['AGE'])
    print('A batch of targets:', label_batch )

Every feature: ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
A batch of ages: tf.Tensor([41 33 29 33 36], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int32)


### Next step is to select the features with corresponding data type

In [6]:
feature_columns = []

# Numeric columns
numerics = [
    'LIMIT_BAL', 
    'AGE',
    'BILL_AMT1', 
    'BILL_AMT2', 
    'BILL_AMT3', 
    'BILL_AMT4', 
    'BILL_AMT5', 
    'BILL_AMT6', 
    'PAY_AMT1', 
    'PAY_AMT2', 
    'PAY_AMT3', 
    'PAY_AMT4', 
    'PAY_AMT5', 
    'PAY_AMT6'
]
for header in numerics:
    feature_columns.append(tf.feature_column.numeric_column(header))

# Categorical columns
categoricals = [
    'SEX', 
    'EDUCATION', 
    'MARRIAGE', 
    'PAY_0', 
    'PAY_2', 
    'PAY_3', 
    'PAY_4', 
    'PAY_5', 
    'PAY_6'
]

# Since the categorical variables take so many levles, we need to use embedding 
# to reduce the dimensionality

dimension = [
    2,
    3,
    3,
    15,
    15,
    15,
    15,
    15,
    15
]
i = 0
for header in categoricals:
    category = tf.feature_column.categorical_column_with_identity(
        header, len(train[header].unique()), default_value=0
    )
    category_one_hot = tf.feature_column.indicator_column(category)
    feature_columns.append(category_one_hot)
    
#     Now let's add the emedding layer
    category_embedding = tf.feature_column.embedding_column(category, dimension=dimension[i])
    feature_columns.append(category_embedding)
    i += 1

In [7]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [8]:
batch_size = 64
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [9]:
model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(64, activation='softmax'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, clipnorm=1.)
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds, validation_data=val_ds, epochs=3)


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/3
338/338 [==============================] - 27s 80ms/step - loss: 0.5388 - accuracy: 0.7661 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
338/338 [==============================] - 29s 85ms/step - loss: 0.5304 - accuracy: 0.7773 - val_loss: 0.5453 - val_accuracy: 0.7667
Epoch 3/3
338/338 [==============================] - 28s 82ms/step - loss: 0.5307 - accuracy: 0.7773 - val_loss: 0.5447 - val_accuracy: 0.7667


In [10]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

94/94 [==============================] - 7s 73ms/step - loss: 0.5159 - accuracy: 0.7890
Accuracy 0.789


### Now let's try to improve the model
3 different adjustment will be made:
1. 'AGE' will be transformed into bucket variable, since it makes more sense to treat them like people of different age group.
2. Try to standardize the numeric variables.
3. Do a PCA to the numerics and see if we can trhow away some variables.

In [11]:
# Firstly, 'AGE' as bucket input

feature_columns = []

age = tf.feature_column.numeric_column("AGE")

age_buckets = tf.feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

In [12]:
# Numeric columns to be standardised
raw_numerics = [
    'AGE',
    'LIMIT_BAL', 
    'BILL_AMT1', 
    'BILL_AMT2', 
    'BILL_AMT3', 
    'BILL_AMT4', 
    'BILL_AMT5', 
    'BILL_AMT6', 
    'PAY_AMT1', 
    'PAY_AMT2', 
    'PAY_AMT3', 
    'PAY_AMT4', 
    'PAY_AMT5', 
    'PAY_AMT6'
]

# 9 principal components are kept after PCA
principalColumns = [
    'PC1',
    'PC2',
    'PC3',
    'PC4',
    'PC5',
    'PC6',
    'PC7',
    'PC8', 
    'PC9',
]

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
pca = PCA(0.95)

# Firstly we fit only the train data
scaler.fit(train.loc[:, raw_numerics])

# Then we apply the transformation to all the datasets
train_img = scaler.transform(train.loc[:, raw_numerics])
test_img = scaler.transform(test.loc[:, raw_numerics])
val_img = scaler.transform(val.loc[:, raw_numerics])

pca.fit(train_img)


train_img = pca.transform(train_img)
test_img = pca.transform(test_img)
val_img = pca.transform(val_img)

principalTrain = pd.DataFrame(data=train_img,
                             columns=principalColumns,
                             index=train.index)
principalTest = pd.DataFrame(data=test_img,
                            columns=principalColumns,
                            index=test.index)
principalVal = pd.DataFrame(data=val_img,
                           columns=principalColumns,
                           index=val.index)


train = pd.concat([principalTrain, train], axis=1)
test = pd.concat([principalTest, test], axis=1)
val = pd.concat([principalVal, val], axis=1)

for header in principalColumns:
    feature_columns.append(tf.feature_column.numeric_column(header))
    
# Categorical columns
categoricals = [
    'SEX', 
    'EDUCATION', 
    'MARRIAGE', 
    'PAY_0', 
    'PAY_2', 
    'PAY_3', 
    'PAY_4', 
    'PAY_5', 
    'PAY_6'
]

# Since the categorical variables take so many levles, we need to use embedding 
# to reduce the dimensionality

dimension = [
    4,
    10,
    6,
    15,
    15,
    15,
    15,
    15,
    15
]
i = 0
for header in categoricals:
    category = tf.feature_column.categorical_column_with_identity(
        header, len(train[header].unique()), default_value=0
    )
    category_one_hot = tf.feature_column.indicator_column(category)
    feature_columns.append(category_one_hot)
    
#     Now let's add the emedding layer
    category_embedding = tf.feature_column.embedding_column(category, dimension=dimension[i])
    feature_columns.append(category_embedding)
    i += 1

In [13]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [14]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [15]:
model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='softmax'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True)
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds, validation_data=val_ds, epochs=5)

Epoch 1/5
675/675 [==============================] - 63s 93ms/step - loss: 0.4609 - accuracy: 0.8058 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
675/675 [==============================] - 69s 103ms/step - loss: 0.4379 - accuracy: 0.8192 - val_loss: 0.4628 - val_accuracy: 0.8017
Epoch 3/5
675/675 [==============================] - 66s 98ms/step - loss: 0.4342 - accuracy: 0.8202 - val_loss: 0.4620 - val_accuracy: 0.8017
Epoch 4/5
675/675 [==============================] - 70s 103ms/step - loss: 0.4328 - accuracy: 0.8205 - val_loss: 0.4599 - val_accuracy: 0.8029
Epoch 5/5
675/675 [==============================] - 60s 89ms/step - loss: 0.4298 - accuracy: 0.8217 - val_loss: 0.4631 - val_accuracy: 0.7983


In [16]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

188/188 [==============================] - 10s 54ms/step - loss: 0.4139 - accuracy: 0.8325
Accuracy 0.8325
